In [57]:
# imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime
import pickle
import pickle5
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.metrics import r2_score   
from sklearn import metrics
from sklearn.preprocessing import StandardScaler
import folium
from sklearn.ensemble import RandomForestRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV, cross_val_score, cross_val_predict
from sklearn.ensemble import GradientBoostingRegressor

## block warning
import warnings
warnings.filterwarnings('ignore')

In [58]:
with open("../data/ut_V6_classification-4H.pkl", "rb") as fh:
    df = pickle5.load(fh)
    
df.head()

,life_time,datetime_start,datetime_end,latitude,longitude,distance_center_km,weekday_b,charging_ports,lag1,lag2,...,hour_b_22,hour_b_23,Station_Name_BOULDER / N BOULDER REC 1,Station_Name_COMM VITALITY / 1000WALNUT,Station_Name_COMM VITALITY / 1104 SPRUCE1,4Hsplit,y_pred,Station_Name,weekday_b_name,tod
3734,118.5,2018-01-05 14:25:00,2018-01-05 16:23:30,40.000148,-105.282437,2.096847,4,2,1271.5,135.5,...,0,0,0,0,0,0,1,BOULDER / BASELINE ST1,Friday,Midday
3735,886.5,2018-01-05 17:02:00,2018-01-06 07:48:30,40.000148,-105.282437,2.096847,4,2,118.5,1271.5,...,0,0,0,0,0,1,1,BOULDER / BASELINE ST1,Friday,Afternoon
3736,326.5,2018-01-06 09:28:00,2018-01-06 14:54:30,40.000148,-105.282437,2.096847,5,2,886.5,118.5,...,0,0,0,0,0,1,0,BOULDER / BASELINE ST1,Saturday,Morning
3737,996.5,2018-01-06 17:09:00,2018-01-07 09:45:30,40.000148,-105.282437,2.096847,5,2,326.5,886.5,...,0,0,0,0,0,1,1,BOULDER / BASELINE ST1,Saturday,Afternoon
3738,52.5,2018-01-07 10:58:00,2018-01-07 11:50:30,40.000148,-105.282437,2.096847,6,2,996.5,326.5,...,0,0,0,0,0,0,0,BOULDER / BASELINE ST1,Sunday,Morning


In [59]:
df.shape

(5110, 102)

In [60]:
df.groupby(['y_pred']).count().life_time

y_pred
0    3062
1    2048
Name: life_time, dtype: int64

# Modeling

In [61]:
def mean_absolute_percentage_error(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true))*100

In [62]:
## To test model
def test_model(y_test, y_pred):
    MAE = metrics.mean_absolute_error(y_test, y_pred)
    #print('MAE (Mean Absolute Error):', MAE)
    MSE = metrics.mean_squared_error(y_test, y_pred)
    #print('MSE (Mean Squared Error):', MSE)
    RMSE = np.sqrt(metrics.mean_squared_error(y_test, y_pred))
    #print('RMSE (Root Mean Squared Error):', RMSE)
    MAPE = mean_absolute_percentage_error(y_test, y_pred)
    
    NRMSE = RMSE/np.mean(y_test)
    return MAE, RMSE, MSE, MAPE, NRMSE

In [65]:
weekdays = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
positions = [0,1,2,3,4,5,6]

To make coding easier the models are renamed:
- M1: Baseline
- M5: Full

In [67]:
## M1
features1 = ['weekday_b_name_Monday', 'weekday_b_name_Saturday',
             'weekday_b_name_Sunday', 'weekday_b_name_Thursday',
             'weekday_b_name_Tuesday', 'weekday_b_name_Wednesday',
             'tod_Evening','tod_Midday', 'tod_Morning',
             'Station_Name_BOULDER / N BOULDER REC 1',
             'Station_Name_COMM VITALITY / 1000WALNUT',
             'Station_Name_COMM VITALITY / 1104 SPRUCE1'] # dow + tod

## M5
features5_cat = ['weekday_b_name_Monday', 'weekday_b_name_Saturday',
                 'weekday_b_name_Sunday', 'weekday_b_name_Thursday',
                 'weekday_b_name_Tuesday', 'weekday_b_name_Wednesday',
                 'tod_Evening','tod_Midday', 'tod_Morning',
                 'Station_Name_BOULDER / N BOULDER REC 1',
                 'Station_Name_COMM VITALITY / 1000WALNUT',
                 'Station_Name_COMM VITALITY / 1104 SPRUCE1']
features5_con = ['lag1', 'lag2', 'lag3','near_charge_time_4H', 'near_charge_energy_4H',
                 'charge_time_4H', 'charge_energy_4H','service', 'entertainment', 'food', 
                 'childcare', 'medical', 'education', 'waste-management'] # dow + agg. tod + lag + activ.

In [68]:
split = 0.8

In [69]:
results = {}

In [70]:
train, test = np.split(df, [int(split * len(df))])
print("Training shape:",train.shape)
print("Testing shape:", test.shape)

Training shape: (4088, 102)
Testing shape: (1022, 102)


In [71]:
trainS = train[train['y_pred'] == 0]
trainL = train[train['y_pred'] == 1]

testS = test[test['y_pred'] == 0]
testL = test[test['y_pred'] == 1]

In [72]:
print("SHORT: Train: {} Test: {}".format(trainS.shape[0], testS.shape[0]))
print("LONG: Train: {} Test: {}".format(trainL.shape[0], testL.shape[0]))

SHORT: Train: 2433 Test: 629
LONG: Train: 1655 Test: 393


In [73]:
## Get X - train
# Baseline
X1_S = trainS[features1]
X1_L = trainL[features1]

## Get X - test
# Baseline
X1_S_test = testS[features1]
X1_L_test = testL[features1]

# Full
X5_S = trainS[features5_cat+features5_con]
scaler = StandardScaler()
X5_S[features5_con] = scaler.fit_transform(X5_S[features5_con])
X5_S_test = testS[features5_cat+features5_con]
X5_S_test[features5_con] = scaler.transform(X5_S_test[features5_con])

X5_L = trainL[features5_cat+features5_con]
scaler = StandardScaler()
X5_L[features5_con] = scaler.fit_transform(X5_L[features5_con])
X5_L_test = testL[features5_cat+features5_con]
X5_L_test[features5_con] = scaler.transform(X5_L_test[features5_con])

## Get y - train
y_S = trainS['life_time']
y_L = trainL['life_time']

## Get y - test
y_S_test = testS['life_time']
y_L_test = testL['life_time']

### LINEAR REGRESSION

In [76]:
grid_lr = {'alpha':[0.001,0.01,0.1,0.5,1,3,5,7,10,13,15,20], 'fit_intercept':['True','False']}
results['LR'] = {}

#### Linear regression - model 1

In [79]:
#########################################################
#################### SEPERATE RESULTS ###################

results['LR']['M1'] = {}
results['LR']['M1']['S'] = {}
results['LR']['M1']['L'] = {}
results['LR']['M1']['Com'] = {}

####################### Hyperparameters #######################
## get hyperparameters and fit models
# short
reg = Ridge(random_state=42)
reg_gs = RandomizedSearchCV(reg, grid_lr, random_state=42, cv=5, 
                            n_iter=50, scoring='neg_mean_squared_error')
reg_gs.fit(X1_S,y_S)
paramS = reg_gs.best_params_
print(paramS)
# Fit model
regs = Ridge(random_state=42, alpha=paramS['alpha'], fit_intercept=False)
regs.fit(X1_S,y_S)

# long
reg = Ridge(random_state=42)
reg_gs = RandomizedSearchCV(reg, grid_lr, random_state=42, cv=5, 
                            n_iter=50, scoring='neg_mean_squared_error')
reg_gs.fit(X1_L,y_L)
paramL = reg_gs.best_params_
print(paramL)
# Fit model
regl = Ridge(random_state=42, alpha=paramL['alpha'], fit_intercept=False)
regl.fit(X1_L,y_L)

#################### Predictions and evaluate ####################
########################### short
## Predict
y_test_predS = regs.predict(X1_S_test)
y_train_predS = regs.predict(X1_S)

## Get metrics  
MAE_test, RMSE_test, MSE_test, MAPE_test, NRMSE_test = test_model(y_S_test,y_test_predS)
MAE_train, RMSE_train, MSE_train, MAPE_train, NRMSE_train = test_model(y_S,y_train_predS)

## Save results
results['LR']['M1']['S'] = {'RMSE_train':RMSE_train,'RMSE_test':RMSE_test,
                              'MAE_train': MAE_train,'MAE_test':MAE_test,
                              'rsq_train':r2_score(y_S, y_train_predS), 
                              'rsq_test':r2_score(y_S_test, y_test_predS),
                              'MAPE_test':MAPE_test,'MAPE_train':MAPE_train,
                              'NRMSE_test':NRMSE_test,'NRMSE_train':NRMSE_train}

########################### long
## predict
y_test_predL = regl.predict(X1_L_test)
y_train_predL = regl.predict(X1_L)

## Get metrics  
MAE_test, RMSE_test, MSE_test, MAPE_test, NRMSE_test = test_model(y_L_test,y_test_predL)
MAE_train, RMSE_train, MSE_train, MAPE_train, NRMSE_train = test_model(y_L,y_train_predL)

## Save results
results['LR']['M1']['L'] = {'RMSE_train':RMSE_train,'RMSE_test':RMSE_test,
                              'MAE_train': MAE_train,'MAE_test':MAE_test,
                              'rsq_train':r2_score(y_L, y_train_predL), 
                              'rsq_test':r2_score(y_L_test, y_test_predL),
                              'MAPE_test':MAPE_test,'MAPE_train':MAPE_train,
                              'NRMSE_test':NRMSE_test,'NRMSE_train':NRMSE_train}

########################### combined
y_com_real_test = pd.concat([y_S_test, y_L_test])
y_com_pred_test = np.concatenate([y_test_predS,y_test_predL])
y_com_real_train = pd.concat([y_S, y_L])
y_com_pred_train = np.concatenate([y_train_predS,y_train_predL])

## Get metrics  
MAE_test, RMSE_test, MSE_test, MAPE_test, NRMSE_test = test_model(y_com_real_test,
                                                                  y_com_pred_test)
MAE_train, RMSE_train, MSE_train, MAPE_train, NRMSE_train = test_model(y_com_real_train,
                                                                       y_com_pred_train)

## Save results
results['LR']['M1']['Com'] = {'RMSE_train':RMSE_train,'RMSE_test':RMSE_test,
                              'MAE_train': MAE_train,'MAE_test':MAE_test,
                              'rsq_train':r2_score(y_com_real_train,y_com_pred_train), 
                              'rsq_test':r2_score(y_com_real_test,y_com_pred_test),
                              'MAPE_test':MAPE_test,'MAPE_train':MAPE_train,
                              'NRMSE_test':NRMSE_test,'NRMSE_train':NRMSE_train}

{'fit_intercept': 'True', 'alpha': 20}
{'fit_intercept': 'True', 'alpha': 15}


#### Linear Regression - M5

In [80]:
#########################################################
#################### SEPERATE RESULTS ###################

results['LR']['M5'] = {}
results['LR']['M5']['S'] = {}
results['LR']['M5']['L'] = {}
results['LR']['M5']['Com'] = {}

####################### Hyperparameters #######################
## get hyperparameters and fit models
# short
reg = Ridge(random_state=42)
reg_gs = RandomizedSearchCV(reg, grid_lr, random_state=42, cv=5, 
                            n_iter=50, scoring='neg_mean_squared_error')
reg_gs.fit(X5_S,y_S)
paramS = reg_gs.best_params_
print(paramS)
# Fit model
regs = Ridge(random_state=42, alpha=paramS['alpha'], fit_intercept=False)
regs.fit(X5_S,y_S)

# long
reg = Ridge(random_state=42)
reg_gs = RandomizedSearchCV(reg, grid_lr, random_state=42, cv=5, 
                            n_iter=50, scoring='neg_mean_squared_error')
reg_gs.fit(X5_L,y_L)
paramL = reg_gs.best_params_
print(paramL)
# Fit model
regl = Ridge(random_state=42, alpha=paramL['alpha'], fit_intercept=False)
regl.fit(X5_L,y_L)

#################### Predictions and evaluate ####################
########################### short
## Predict
y_test_predS = regs.predict(X5_S_test)
y_train_predS = regs.predict(X5_S)

## Get metrics  
MAE_test, RMSE_test, MSE_test, MAPE_test, NRMSE_test = test_model(y_S_test,y_test_predS)
MAE_train, RMSE_train, MSE_train, MAPE_train, NRMSE_train = test_model(y_S,y_train_predS)

## Save results
results['LR']['M5']['S'] = {'RMSE_train':RMSE_train,'RMSE_test':RMSE_test,
                              'MAE_train': MAE_train,'MAE_test':MAE_test,
                              'rsq_train':r2_score(y_S, y_train_predS), 
                              'rsq_test':r2_score(y_S_test, y_test_predS),
                              'MAPE_test':MAPE_test,'MAPE_train':MAPE_train,
                              'NRMSE_test':NRMSE_test,'NRMSE_train':NRMSE_train}

########################### long
## predict
y_test_predL = regl.predict(X5_L_test)
y_train_predL = regl.predict(X5_L)

## Get metrics  
MAE_test, RMSE_test, MSE_test, MAPE_test, NRMSE_test = test_model(y_L_test,y_test_predL)
MAE_train, RMSE_train, MSE_train, MAPE_train, NRMSE_train = test_model(y_L,y_train_predL)

## Save results
results['LR']['M5']['L'] = {'RMSE_train':RMSE_train,'RMSE_test':RMSE_test,
                              'MAE_train': MAE_train,'MAE_test':MAE_test,
                              'rsq_train':r2_score(y_L, y_train_predL), 
                              'rsq_test':r2_score(y_L_test, y_test_predL),
                              'MAPE_test':MAPE_test,'MAPE_train':MAPE_train,
                              'NRMSE_test':NRMSE_test,'NRMSE_train':NRMSE_train}

########################### combined
y_com_real_test = pd.concat([y_S_test, y_L_test])
y_com_pred_test = np.concatenate([y_test_predS,y_test_predL])
y_com_real_train = pd.concat([y_S, y_L])
y_com_pred_train = np.concatenate([y_train_predS,y_train_predL])

## Get metrics  
MAE_test, RMSE_test, MSE_test, MAPE_test, NRMSE_test = test_model(y_com_real_test,
                                                                  y_com_pred_test)
MAE_train, RMSE_train, MSE_train, MAPE_train, NRMSE_train = test_model(y_com_real_train,
                                                                       y_com_pred_train)

## Save results
results['LR']['M5']['Com'] = {'RMSE_train':RMSE_train,'RMSE_test':RMSE_test,
                              'MAE_train': MAE_train,'MAE_test':MAE_test,
                              'rsq_train':r2_score(y_com_real_train,y_com_pred_train), 
                              'rsq_test':r2_score(y_com_real_test,y_com_pred_test),
                              'MAPE_test':MAPE_test,'MAPE_train':MAPE_train,
                              'NRMSE_test':NRMSE_test,'NRMSE_train':NRMSE_train}

{'fit_intercept': 'True', 'alpha': 20}
{'fit_intercept': 'True', 'alpha': 20}


### RANDOM FOREST REGRESSOR

In [81]:
grid_RF = {'n_estimators':[100,300,500,1000], 'min_samples_split':[2,3,5,7],
           'min_samples_leaf':[3,5,10,15], 'max_depth':[2,3,5,7]}
results['RF'] = {}

#### Random forest regressor - model 1

In [82]:
#########################################################
#################### SEPERATE RESULTS ###################

results['RF']['M1'] = {}
results['RF']['M1']['S'] = {}
results['RF']['M1']['L'] = {}
results['RF']['M1']['Com'] = {}

####################### Hyperparameters #######################
## get hyperparameters and fit models
# short
reg = RandomForestRegressor(random_state=42)
reg_gs = RandomizedSearchCV(reg, grid_RF, random_state=42, cv=5, 
                            n_iter=50, scoring='neg_mean_squared_error')
reg_gs.fit(X1_S,y_S)
paramS = reg_gs.best_params_
print(paramS)
# Fit model
regs = RandomForestRegressor(random_state=42, 
                            min_samples_split=paramS['min_samples_split'],
                            min_samples_leaf=paramS['min_samples_leaf'],
                            max_depth=paramS['max_depth'],
                            n_estimators=paramS['n_estimators'])
regs.fit(X1_S,y_S)

# long
reg = RandomForestRegressor(random_state=42)
reg_gs = RandomizedSearchCV(reg, grid_RF, random_state=42, cv=5, 
                            n_iter=50, scoring='neg_mean_squared_error')
reg_gs.fit(X1_L,y_L)
paramL = reg_gs.best_params_
print(paramL)
# Fit model
regl = RandomForestRegressor(random_state=42, 
                            min_samples_split=paramL['min_samples_split'],
                            min_samples_leaf=paramL['min_samples_leaf'], 
                            max_depth=paramL['max_depth'],
                            n_estimators=paramL['n_estimators'])
regl.fit(X1_L,y_L)

#################### Predictions and evaluate ####################
########################### short
## Predict
y_test_predS = regs.predict(X1_S_test)
y_train_predS = regs.predict(X1_S)

## Get metrics  
MAE_test, RMSE_test, MSE_test, MAPE_test, NRMSE_test = test_model(y_S_test,y_test_predS)
MAE_train, RMSE_train, MSE_train, MAPE_train, NRMSE_train = test_model(y_S,y_train_predS)

## Save results
results['RF']['M1']['S'] = {'RMSE_train':RMSE_train,'RMSE_test':RMSE_test,
                              'MAE_train': MAE_train,'MAE_test':MAE_test,
                              'rsq_train':r2_score(y_S, y_train_predS), 
                              'rsq_test':r2_score(y_S_test, y_test_predS),
                              'MAPE_test':MAPE_test,'MAPE_train':MAPE_train,
                              'NRMSE_test':NRMSE_test,'NRMSE_train':NRMSE_train}

########################### long
## predict
y_test_predL = regl.predict(X1_L_test)
y_train_predL = regl.predict(X1_L)

## Get metrics  
MAE_test, RMSE_test, MSE_test, MAPE_test, NRMSE_test = test_model(y_L_test,y_test_predL)
MAE_train, RMSE_train, MSE_train, MAPE_train, NRMSE_train = test_model(y_L,y_train_predL)

## Save results
results['RF']['M1']['L'] = {'RMSE_train':RMSE_train,'RMSE_test':RMSE_test,
                              'MAE_train': MAE_train,'MAE_test':MAE_test,
                              'rsq_train':r2_score(y_L, y_train_predL), 
                              'rsq_test':r2_score(y_L_test, y_test_predL),
                              'MAPE_test':MAPE_test,'MAPE_train':MAPE_train,
                              'NRMSE_test':NRMSE_test,'NRMSE_train':NRMSE_train}

########################### combined
y_com_real_test = pd.concat([y_S_test, y_L_test])
y_com_pred_test = np.concatenate([y_test_predS,y_test_predL])
y_com_real_train = pd.concat([y_S, y_L])
y_com_pred_train = np.concatenate([y_train_predS,y_train_predL])

## Get metrics  
MAE_test, RMSE_test, MSE_test, MAPE_test, NRMSE_test = test_model(y_com_real_test,
                                                                  y_com_pred_test)
MAE_train, RMSE_train, MSE_train, MAPE_train, NRMSE_train = test_model(y_com_real_train,
                                                                       y_com_pred_train)

## Save results
results['RF']['M1']['Com'] = {'RMSE_train':RMSE_train,'RMSE_test':RMSE_test,
                              'MAE_train': MAE_train,'MAE_test':MAE_test,
                              'rsq_train':r2_score(y_com_real_train,y_com_pred_train), 
                              'rsq_test':r2_score(y_com_real_test,y_com_pred_test),
                              'MAPE_test':MAPE_test,'MAPE_train':MAPE_train,
                              'NRMSE_test':NRMSE_test,'NRMSE_train':NRMSE_train}

{'n_estimators': 100, 'min_samples_split': 3, 'min_samples_leaf': 3, 'max_depth': 3}
{'n_estimators': 100, 'min_samples_split': 3, 'min_samples_leaf': 15, 'max_depth': 5}


#### RF - M5

In [83]:
#########################################################
#################### SEPERATE RESULTS ###################

results['RF']['M5'] = {}
results['RF']['M5']['S'] = {}
results['RF']['M5']['L'] = {}
results['RF']['M5']['Com'] = {}

####################### Hyperparameters #######################
## get hyperparameters and fit models
# short
reg = RandomForestRegressor(random_state=42)
reg_gs = RandomizedSearchCV(reg, grid_RF, random_state=42, cv=5, 
                            n_iter=50, scoring='neg_mean_squared_error')
reg_gs.fit(X5_S,y_S)
paramS = reg_gs.best_params_
print(paramS)
# Fit model
regs = RandomForestRegressor(random_state=42, 
                            min_samples_split=paramS['min_samples_split'],
                            min_samples_leaf=paramS['min_samples_leaf'],
                            max_depth=paramS['max_depth'],
                            n_estimators=paramS['n_estimators'])
regs.fit(X5_S,y_S)

# long
reg = RandomForestRegressor(random_state=42)
reg_gs = RandomizedSearchCV(reg, grid_RF, random_state=42, cv=5, 
                            n_iter=50, scoring='neg_mean_squared_error')
reg_gs.fit(X5_L,y_L)
paramL = reg_gs.best_params_
print(paramL)
# Fit model
regl = RandomForestRegressor(random_state=42, 
                            min_samples_split=paramL['min_samples_split'],
                            min_samples_leaf=paramL['min_samples_leaf'], 
                            max_depth=paramL['max_depth'],
                            n_estimators=paramL['n_estimators'])
regl.fit(X5_L,y_L)

#################### Predictions and evaluate ####################
########################### short
## Predict
y_test_predS = regs.predict(X5_S_test)
y_train_predS = regs.predict(X5_S)

## Get metrics  
MAE_test, RMSE_test, MSE_test, MAPE_test, NRMSE_test = test_model(y_S_test,y_test_predS)
MAE_train, RMSE_train, MSE_train, MAPE_train, NRMSE_train = test_model(y_S,y_train_predS)

## Save results
results['RF']['M5']['S'] = {'RMSE_train':RMSE_train,'RMSE_test':RMSE_test,
                              'MAE_train': MAE_train,'MAE_test':MAE_test,
                              'rsq_train':r2_score(y_S, y_train_predS), 
                              'rsq_test':r2_score(y_S_test, y_test_predS),
                              'MAPE_test':MAPE_test,'MAPE_train':MAPE_train,
                              'NRMSE_test':NRMSE_test,'NRMSE_train':NRMSE_train}

########################### long
## predict
y_test_predL = regl.predict(X5_L_test)
y_train_predL = regl.predict(X5_L)

## Get metrics  
MAE_test, RMSE_test, MSE_test, MAPE_test, NRMSE_test = test_model(y_L_test,y_test_predL)
MAE_train, RMSE_train, MSE_train, MAPE_train, NRMSE_train = test_model(y_L,y_train_predL)

## Save results
results['RF']['M5']['L'] = {'RMSE_train':RMSE_train,'RMSE_test':RMSE_test,
                              'MAE_train': MAE_train,'MAE_test':MAE_test,
                              'rsq_train':r2_score(y_L, y_train_predL), 
                              'rsq_test':r2_score(y_L_test, y_test_predL),
                              'MAPE_test':MAPE_test,'MAPE_train':MAPE_train,
                              'NRMSE_test':NRMSE_test,'NRMSE_train':NRMSE_train}

########################### combined
y_com_real_test = pd.concat([y_S_test, y_L_test])
y_com_pred_test = np.concatenate([y_test_predS,y_test_predL])
y_com_real_train = pd.concat([y_S, y_L])
y_com_pred_train = np.concatenate([y_train_predS,y_train_predL])

## Get metrics  
MAE_test, RMSE_test, MSE_test, MAPE_test, NRMSE_test = test_model(y_com_real_test,
                                                                  y_com_pred_test)
MAE_train, RMSE_train, MSE_train, MAPE_train, NRMSE_train = test_model(y_com_real_train,
                                                                       y_com_pred_train)

## Save results
results['RF']['M5']['Com'] = {'RMSE_train':RMSE_train,'RMSE_test':RMSE_test,
                              'MAE_train': MAE_train,'MAE_test':MAE_test,
                              'rsq_train':r2_score(y_com_real_train,y_com_pred_train), 
                              'rsq_test':r2_score(y_com_real_test,y_com_pred_test),
                              'MAPE_test':MAPE_test,'MAPE_train':MAPE_train,
                              'NRMSE_test':NRMSE_test,'NRMSE_train':NRMSE_train}

{'n_estimators': 500, 'min_samples_split': 2, 'min_samples_leaf': 15, 'max_depth': 7}
{'n_estimators': 100, 'min_samples_split': 5, 'min_samples_leaf': 15, 'max_depth': 3}


### MLP

In [84]:
grid_MLP = {'alpha':[0.0001,0.001,0.1,1,2,5,10], 'hidden_layer_sizes':
            [(32),(32,32),(32,32),(32,32,32,32),(64),(64,64),(64,64,64),(64,64,64,64),
             (128),(128,128),(128,128,128),(128,128,128,128),(256),(256,256),(256,256,256),
             (256,256,256,256)],
            'activation': ['tanh', 'relu', 'logistic']}
results['MLP'] = {}

#### MLP - Model 1

In [85]:
#########################################################
#################### SEPERATE RESULTS ###################

results['MLP']['M1'] = {}
results['MLP']['M1']['S'] = {}
results['MLP']['M1']['L'] = {}
results['MLP']['M1']['Com'] = {}

####################### Hyperparameters #######################
## get hyperparameters and fit models
## short
reg = MLPRegressor(random_state=42, max_iter=500, early_stopping=True)
reg_gs = RandomizedSearchCV(reg, grid_MLP, random_state=42, cv=5, 
                            n_iter=50, scoring='neg_mean_squared_error')
reg_gs.fit(X1_S,y_S)
paramS = reg_gs.best_params_
print(paramS)
# Fit model
regs = MLPRegressor(random_state=42, alpha=paramS['alpha'], 
                    hidden_layer_sizes=paramS['hidden_layer_sizes'],
                    max_iter=500, early_stopping=True)
regs.fit(X1_S,y_S)

## long
reg = MLPRegressor(random_state=42, max_iter=500, early_stopping=True)
reg_gs = RandomizedSearchCV(reg, grid_MLP, random_state=42, cv=5, 
                            n_iter=50, scoring='neg_mean_squared_error')
reg_gs.fit(X1_L,y_L)
paramL = reg_gs.best_params_
print(paramL)
# Fit model
regl = MLPRegressor(random_state=42, alpha=paramS['alpha'], 
                    hidden_layer_sizes=paramS['hidden_layer_sizes'],
                    max_iter=500, early_stopping=True)
regl.fit(X1_L,y_L)


#################### Predictions and evaluate ####################
########################### short
## Predict
y_test_predS = regs.predict(X1_S_test)
y_train_predS = regs.predict(X1_S)

## Get metrics  
MAE_test, RMSE_test, MSE_test, MAPE_test, NRMSE_test = test_model(y_S_test,y_test_predS)
MAE_train, RMSE_train, MSE_train, MAPE_train, NRMSE_train = test_model(y_S,y_train_predS)

## Save results
results['MLP']['M1']['S'] = {'RMSE_train':RMSE_train,'RMSE_test':RMSE_test,
                              'MAE_train': MAE_train,'MAE_test':MAE_test,
                              'rsq_train':r2_score(y_S, y_train_predS), 
                              'rsq_test':r2_score(y_S_test, y_test_predS),
                              'MAPE_test':MAPE_test,'MAPE_train':MAPE_train,
                              'NRMSE_test':NRMSE_test,'NRMSE_train':NRMSE_train}

########################### long
## predict
y_test_predL = regl.predict(X1_L_test)
y_train_predL = regl.predict(X1_L)

## Get metrics  
MAE_test, RMSE_test, MSE_test, MAPE_test, NRMSE_test = test_model(y_L_test,y_test_predL)
MAE_train, RMSE_train, MSE_train, MAPE_train, NRMSE_train = test_model(y_L,y_train_predL)

## Save results
results['MLP']['M1']['L'] = {'RMSE_train':RMSE_train,'RMSE_test':RMSE_test,
                              'MAE_train': MAE_train,'MAE_test':MAE_test,
                              'rsq_train':r2_score(y_L, y_train_predL), 
                              'rsq_test':r2_score(y_L_test, y_test_predL),
                              'MAPE_test':MAPE_test,'MAPE_train':MAPE_train,
                              'NRMSE_test':NRMSE_test,'NRMSE_train':NRMSE_train}

########################### combined
y_com_real_test = pd.concat([y_S_test, y_L_test])
y_com_pred_test = np.concatenate([y_test_predS,y_test_predL])
y_com_real_train = pd.concat([y_S, y_L])
y_com_pred_train = np.concatenate([y_train_predS,y_train_predL])

## Get metrics  
MAE_test, RMSE_test, MSE_test, MAPE_test, NRMSE_test = test_model(y_com_real_test,
                                                                  y_com_pred_test)
MAE_train, RMSE_train, MSE_train, MAPE_train, NRMSE_train = test_model(y_com_real_train,
                                                                       y_com_pred_train)

## Save results
results['MLP']['M1']['Com'] = {'RMSE_train':RMSE_train,'RMSE_test':RMSE_test,
                              'MAE_train': MAE_train,'MAE_test':MAE_test,
                              'rsq_train':r2_score(y_com_real_train,y_com_pred_train), 
                              'rsq_test':r2_score(y_com_real_test,y_com_pred_test),
                              'MAPE_test':MAPE_test,'MAPE_train':MAPE_train,
                              'NRMSE_test':NRMSE_test,'NRMSE_train':NRMSE_train}

{'hidden_layer_sizes': (256, 256, 256), 'alpha': 0.1, 'activation': 'relu'}
{'hidden_layer_sizes': (256, 256, 256), 'alpha': 0.1, 'activation': 'relu'}


#### MLP - M5

In [86]:
#########################################################
#################### SEPERATE RESULTS ###################

results['MLP']['M5'] = {}
results['MLP']['M5']['S'] = {}
results['MLP']['M5']['L'] = {}
results['MLP']['M5']['Com'] = {}

####################### Hyperparameters #######################
## get hyperparameters and fit models
## short
reg = MLPRegressor(random_state=42, max_iter=500, early_stopping=True)
reg_gs = RandomizedSearchCV(reg, grid_MLP, random_state=42, cv=5, 
                            n_iter=50, scoring='neg_mean_squared_error')
reg_gs.fit(X5_S,y_S)
paramS = reg_gs.best_params_
print(paramS)
# Fit model
regs = MLPRegressor(random_state=42, alpha=paramS['alpha'], 
                    hidden_layer_sizes=paramS['hidden_layer_sizes'],
                    max_iter=500, early_stopping=True)
regs.fit(X5_S,y_S)

## long
reg = MLPRegressor(random_state=42, max_iter=500, early_stopping=True)
reg_gs = RandomizedSearchCV(reg, grid_MLP, random_state=42, cv=5, 
                            n_iter=50, scoring='neg_mean_squared_error')
reg_gs.fit(X5_L,y_L)
paramL = reg_gs.best_params_
print(paramL)
# Fit model
regl = MLPRegressor(random_state=42, alpha=paramS['alpha'], 
                    hidden_layer_sizes=paramS['hidden_layer_sizes'],
                    max_iter=500, early_stopping=True)
regl.fit(X5_L,y_L)


#################### Predictions and evaluate ####################
########################### short
## Predict
y_test_predS = regs.predict(X5_S_test)
y_train_predS = regs.predict(X5_S)

## Get metrics  
MAE_test, RMSE_test, MSE_test, MAPE_test, NRMSE_test = test_model(y_S_test,y_test_predS)
MAE_train, RMSE_train, MSE_train, MAPE_train, NRMSE_train = test_model(y_S,y_train_predS)

## Save results
results['MLP']['M5']['S'] = {'RMSE_train':RMSE_train,'RMSE_test':RMSE_test,
                              'MAE_train': MAE_train,'MAE_test':MAE_test,
                              'rsq_train':r2_score(y_S, y_train_predS), 
                              'rsq_test':r2_score(y_S_test, y_test_predS),
                              'MAPE_test':MAPE_test,'MAPE_train':MAPE_train,
                              'NRMSE_test':NRMSE_test,'NRMSE_train':NRMSE_train}

########################### long
## predict
y_test_predL = regl.predict(X5_L_test)
y_train_predL = regl.predict(X5_L)

## Get metrics  
MAE_test, RMSE_test, MSE_test, MAPE_test, NRMSE_test = test_model(y_L_test,y_test_predL)
MAE_train, RMSE_train, MSE_train, MAPE_train, NRMSE_train = test_model(y_L,y_train_predL)

## Save results
results['MLP']['M5']['L'] = {'RMSE_train':RMSE_train,'RMSE_test':RMSE_test,
                              'MAE_train': MAE_train,'MAE_test':MAE_test,
                              'rsq_train':r2_score(y_L, y_train_predL), 
                              'rsq_test':r2_score(y_L_test, y_test_predL),
                              'MAPE_test':MAPE_test,'MAPE_train':MAPE_train,
                              'NRMSE_test':NRMSE_test,'NRMSE_train':NRMSE_train}

########################### combined
y_com_real_test = pd.concat([y_S_test, y_L_test])
y_com_pred_test = np.concatenate([y_test_predS,y_test_predL])
y_com_real_train = pd.concat([y_S, y_L])
y_com_pred_train = np.concatenate([y_train_predS,y_train_predL])

## Get metrics  
MAE_test, RMSE_test, MSE_test, MAPE_test, NRMSE_test = test_model(y_com_real_test,
                                                                  y_com_pred_test)
MAE_train, RMSE_train, MSE_train, MAPE_train, NRMSE_train = test_model(y_com_real_train,
                                                                       y_com_pred_train)

## Save results
results['MLP']['M5']['Com'] = {'RMSE_train':RMSE_train,'RMSE_test':RMSE_test,
                              'MAE_train': MAE_train,'MAE_test':MAE_test,
                              'rsq_train':r2_score(y_com_real_train,y_com_pred_train), 
                              'rsq_test':r2_score(y_com_real_test,y_com_pred_test),
                              'MAPE_test':MAPE_test,'MAPE_train':MAPE_train,
                              'NRMSE_test':NRMSE_test,'NRMSE_train':NRMSE_train}

{'hidden_layer_sizes': 256, 'alpha': 10, 'activation': 'relu'}
{'hidden_layer_sizes': (32, 32), 'alpha': 0.0001, 'activation': 'relu'}


### GRADIANT BOOST

In [87]:
grid_GB = {'max_depth':[2,3,5,10],'n_estimators':[100,300,500,1000], 
           'learning_rate':[0.001,0.01,0.1,0.5,0.9], 'min_samples_split':[2,3,5,7],
           'min_samples_leaf':[3,5,10,15]}
results['GB'] = {}

#### Gradiant boosting - M1

In [88]:
#########################################################
#################### SEPERATE RESULTS ###################

results['GB']['M1'] = {}
results['GB']['M1']['S'] = {}
results['GB']['M1']['L'] = {}
results['GB']['M1']['Com'] = {}

####################### Hyperparameters #######################
## get hyperparameters
# Short
reg = GradientBoostingRegressor(random_state=42)
reg_gs = RandomizedSearchCV(reg, grid_GB, random_state=42, cv=5, 
                            n_iter=50, scoring='neg_mean_squared_error')
reg_gs.fit(X1_S,y_S)
paramS = reg_gs.best_params_
print(paramS)
# Fit model
regs = GradientBoostingRegressor(random_state=42, 
                                max_depth=paramS['max_depth'],
                                n_estimators=paramS['n_estimators'], 
                                learning_rate=paramS['learning_rate'])
regs.fit(X1_S,y_S)
# Long
reg = GradientBoostingRegressor(random_state=42)
reg_gs = RandomizedSearchCV(reg, grid_GB, random_state=42, cv=5, 
                            n_iter=50, scoring='neg_mean_squared_error')
reg_gs.fit(X1_L,y_L)
paramL = reg_gs.best_params_
print(paramL)
regl = GradientBoostingRegressor(random_state=42, 
                                max_depth=paramL['max_depth'],
                                n_estimators=paramL['n_estimators'], 
                                learning_rate=paramL['learning_rate'])
regl.fit(X1_L,y_L)


#################### Predictions and evaluate ####################
########################### short
## Predict
y_test_predS = regs.predict(X1_S_test)
y_train_predS = regs.predict(X1_S)

## Get metrics  
MAE_test, RMSE_test, MSE_test, MAPE_test, NRMSE_test = test_model(y_S_test,y_test_predS)
MAE_train, RMSE_train, MSE_train, MAPE_train, NRMSE_train = test_model(y_S,y_train_predS)

## Save results
results['GB']['M1']['S'] = {'RMSE_train':RMSE_train,'RMSE_test':RMSE_test,
                              'MAE_train': MAE_train,'MAE_test':MAE_test,
                              'rsq_train':r2_score(y_S, y_train_predS), 
                              'rsq_test':r2_score(y_S_test, y_test_predS),
                              'MAPE_test':MAPE_test,'MAPE_train':MAPE_train,
                              'NRMSE_test':NRMSE_test,'NRMSE_train':NRMSE_train}

########################### long
## predict
y_test_predL = regl.predict(X1_L_test)
y_train_predL = regl.predict(X1_L)

## Get metrics  
MAE_test, RMSE_test, MSE_test, MAPE_test, NRMSE_test = test_model(y_L_test,y_test_predL)
MAE_train, RMSE_train, MSE_train, MAPE_train, NRMSE_train = test_model(y_L,y_train_predL)

## Save results
results['GB']['M1']['L'] = {'RMSE_train':RMSE_train,'RMSE_test':RMSE_test,
                              'MAE_train': MAE_train,'MAE_test':MAE_test,
                              'rsq_train':r2_score(y_L, y_train_predL), 
                              'rsq_test':r2_score(y_L_test, y_test_predL),
                              'MAPE_test':MAPE_test,'MAPE_train':MAPE_train,
                              'NRMSE_test':NRMSE_test,'NRMSE_train':NRMSE_train}

########################### combined
y_com_real_test = pd.concat([y_S_test, y_L_test])
y_com_pred_test = np.concatenate([y_test_predS,y_test_predL])
y_com_real_train = pd.concat([y_S, y_L])
y_com_pred_train = np.concatenate([y_train_predS,y_train_predL])

## Get metrics  
MAE_test, RMSE_test, MSE_test, MAPE_test, NRMSE_test = test_model(y_com_real_test,
                                                                  y_com_pred_test)
MAE_train, RMSE_train, MSE_train, MAPE_train, NRMSE_train = test_model(y_com_real_train,
                                                                       y_com_pred_train)

## Save results
results['GB']['M1']['Com'] = {'RMSE_train':RMSE_train,'RMSE_test':RMSE_test,
                              'MAE_train': MAE_train,'MAE_test':MAE_test,
                              'rsq_train':r2_score(y_com_real_train,y_com_pred_train), 
                              'rsq_test':r2_score(y_com_real_test,y_com_pred_test),
                              'MAPE_test':MAPE_test,'MAPE_train':MAPE_train,
                              'NRMSE_test':NRMSE_test,'NRMSE_train':NRMSE_train}

{'n_estimators': 100, 'min_samples_split': 7, 'min_samples_leaf': 15, 'max_depth': 2, 'learning_rate': 0.1}
{'n_estimators': 300, 'min_samples_split': 7, 'min_samples_leaf': 10, 'max_depth': 3, 'learning_rate': 0.01}


#### GB - M5

In [89]:
#########################################################
#################### SEPERATE RESULTS ###################

results['GB']['M5'] = {}
results['GB']['M5']['S'] = {}
results['GB']['M5']['L'] = {}
results['GB']['M5']['Com'] = {}

####################### Hyperparameters #######################
## get hyperparameters
# Short
reg = GradientBoostingRegressor(random_state=42)
reg_gs = RandomizedSearchCV(reg, grid_GB, random_state=42, cv=5, 
                            n_iter=50, scoring='neg_mean_squared_error')
reg_gs.fit(X5_S,y_S)
paramS = reg_gs.best_params_
print(paramS)
# Fit model
regs = GradientBoostingRegressor(random_state=42, 
                                max_depth=paramS['max_depth'],
                                n_estimators=paramS['n_estimators'], 
                                learning_rate=paramS['learning_rate'])
regs.fit(X5_S,y_S)
# Long
reg = GradientBoostingRegressor(random_state=42)
reg_gs = RandomizedSearchCV(reg, grid_GB, random_state=42, cv=5, 
                            n_iter=50, scoring='neg_mean_squared_error')
reg_gs.fit(X5_L,y_L)
paramL = reg_gs.best_params_
print(paramL)
regl = GradientBoostingRegressor(random_state=42, 
                                max_depth=paramL['max_depth'],
                                n_estimators=paramL['n_estimators'], 
                                learning_rate=paramL['learning_rate'])
regl.fit(X5_L,y_L)


#################### Predictions and evaluate ####################
########################### short
## Predict
y_test_predS = regs.predict(X5_S_test)
y_train_predS = regs.predict(X5_S)

## Get metrics  
MAE_test, RMSE_test, MSE_test, MAPE_test, NRMSE_test = test_model(y_S_test,y_test_predS)
MAE_train, RMSE_train, MSE_train, MAPE_train, NRMSE_train = test_model(y_S,y_train_predS)

## Save results
results['GB']['M5']['S'] = {'RMSE_train':RMSE_train,'RMSE_test':RMSE_test,
                              'MAE_train': MAE_train,'MAE_test':MAE_test,
                              'rsq_train':r2_score(y_S, y_train_predS), 
                              'rsq_test':r2_score(y_S_test, y_test_predS),
                              'MAPE_test':MAPE_test,'MAPE_train':MAPE_train,
                              'NRMSE_test':NRMSE_test,'NRMSE_train':NRMSE_train}

########################### long
## predict
y_test_predL = regl.predict(X5_L_test)
y_train_predL = regl.predict(X5_L)

## Get metrics  
MAE_test, RMSE_test, MSE_test, MAPE_test, NRMSE_test = test_model(y_L_test,y_test_predL)
MAE_train, RMSE_train, MSE_train, MAPE_train, NRMSE_train = test_model(y_L,y_train_predL)

## Save results
results['GB']['M5']['L'] = {'RMSE_train':RMSE_train,'RMSE_test':RMSE_test,
                              'MAE_train': MAE_train,'MAE_test':MAE_test,
                              'rsq_train':r2_score(y_L, y_train_predL), 
                              'rsq_test':r2_score(y_L_test, y_test_predL),
                              'MAPE_test':MAPE_test,'MAPE_train':MAPE_train,
                              'NRMSE_test':NRMSE_test,'NRMSE_train':NRMSE_train}

########################### combined
y_com_real_test = pd.concat([y_S_test, y_L_test])
y_com_pred_test = np.concatenate([y_test_predS,y_test_predL])
y_com_real_train = pd.concat([y_S, y_L])
y_com_pred_train = np.concatenate([y_train_predS,y_train_predL])

## Get metrics  
MAE_test, RMSE_test, MSE_test, MAPE_test, NRMSE_test = test_model(y_com_real_test,
                                                                  y_com_pred_test)
MAE_train, RMSE_train, MSE_train, MAPE_train, NRMSE_train = test_model(y_com_real_train,
                                                                       y_com_pred_train)

## Save results
results['GB']['M5']['Com'] = {'RMSE_train':RMSE_train,'RMSE_test':RMSE_test,
                              'MAE_train': MAE_train,'MAE_test':MAE_test,
                              'rsq_train':r2_score(y_com_real_train,y_com_pred_train), 
                              'rsq_test':r2_score(y_com_real_test,y_com_pred_test),
                              'MAPE_test':MAPE_test,'MAPE_train':MAPE_train,
                              'NRMSE_test':NRMSE_test,'NRMSE_train':NRMSE_train}

{'n_estimators': 300, 'min_samples_split': 7, 'min_samples_leaf': 10, 'max_depth': 3, 'learning_rate': 0.01}
{'n_estimators': 300, 'min_samples_split': 5, 'min_samples_leaf': 10, 'max_depth': 2, 'learning_rate': 0.01}


## Results

In [92]:
methods = ['LR', 'RF', 'MLP', 'GB']
models = ['M1','M5']

print(20*'-',"SHORT",20*'-')
for me in methods:
    for mo in models:
        r = results[me][mo]['S']
        RMSEtest = r['RMSE_test']
        R2test = r['rsq_test']
        MAEtest = r['MAE_test']
        MAPE = r['MAPE_test']
        NRMSE = r['NRMSE_test']
        
        print("{} - {}:\n R2: {} RMSE: {} MAE: {} MAPE: {} NRMSE: {}".format(me,mo,
                                                                              round(np.mean(R2test),2),
                                                                              round(np.mean(RMSEtest),0),
                                                                              round(np.mean(MAEtest),0),
                                                                              round(np.mean(MAPE),0),
                                                                              round(np.mean(NRMSE),2)))        
    print("\n")

-------------------- SHORT --------------------
LR - M1:
 R2: -0.05 RMSE: 409.0 MAE: 309.0 MAPE: 1931.0 NRMSE: 1.45
LR - M5:
 R2: 0.0 RMSE: 399.0 MAE: 312.0 MAPE: 2025.0 NRMSE: 1.41


RF - M1:
 R2: -0.0 RMSE: 399.0 MAE: 311.0 MAPE: 2000.0 NRMSE: 1.41
RF - M5:
 R2: -0.03 RMSE: 405.0 MAE: 323.0 MAPE: 2224.0 NRMSE: 1.43


MLP - M1:
 R2: 0.01 RMSE: 397.0 MAE: 310.0 MAPE: 2117.0 NRMSE: 1.41
MLP - M5:
 R2: 0.01 RMSE: 397.0 MAE: 308.0 MAPE: 2011.0 NRMSE: 1.41


GB - M1:
 R2: -0.0 RMSE: 399.0 MAE: 308.0 MAPE: 1994.0 NRMSE: 1.41
GB - M5:
 R2: -0.02 RMSE: 403.0 MAE: 316.0 MAPE: 2113.0 NRMSE: 1.43




In [93]:
methods = ['LR', 'RF', 'MLP', 'GB']
models = ['M1','M5']

print(20*'-',"LONG",20*'-')
for me in methods:
    for mo in models:
        r = results[me][mo]['L']
        RMSEtest = r['RMSE_test']
        R2test = r['rsq_test']
        MAEtest = r['MAE_test']
        MAPE = r['MAPE_test']
        NRMSE = r['NRMSE_test']
        
        print("{} - {}:\n R2: {} RMSE: {} MAE: {} MAPE: {} NRMSE: {}".format(me,mo,
                                                                              round(np.mean(R2test),2),
                                                                              round(np.mean(RMSEtest),0),
                                                                              round(np.mean(MAEtest),0),
                                                                              round(np.mean(MAPE),0),
                                                                              round(np.mean(NRMSE),2)))        
    print("\n")

-------------------- LONG --------------------
LR - M1:
 R2: -0.32 RMSE: 520.0 MAE: 397.0 MAPE: 1361.0 NRMSE: 0.69
LR - M5:
 R2: -0.0 RMSE: 453.0 MAE: 338.0 MAPE: 1520.0 NRMSE: 0.61


RF - M1:
 R2: 0.04 RMSE: 444.0 MAE: 326.0 MAPE: 1375.0 NRMSE: 0.59
RF - M5:
 R2: 0.05 RMSE: 442.0 MAE: 322.0 MAPE: 1464.0 NRMSE: 0.59


MLP - M1:
 R2: 0.02 RMSE: 448.0 MAE: 329.0 MAPE: 1379.0 NRMSE: 0.6
MLP - M5:
 R2: 0.03 RMSE: 446.0 MAE: 327.0 MAPE: 1451.0 NRMSE: 0.6


GB - M1:
 R2: 0.04 RMSE: 442.0 MAE: 324.0 MAPE: 1423.0 NRMSE: 0.59
GB - M5:
 R2: 0.04 RMSE: 445.0 MAE: 322.0 MAPE: 1509.0 NRMSE: 0.59




In [94]:
methods = ['LR', 'RF', 'MLP', 'GB']
models = ['M1','M5']

print(20*'-',"COMBINED",20*'-')
for me in methods:
    for mo in models:
        r = results[me][mo]['Com']
        RMSEtest = r['RMSE_test']
        R2test = r['rsq_test']
        MAEtest = r['MAE_test']
        MAPE = r['MAPE_test']
        NRMSE = r['NRMSE_test']
        
        print("{} - {}:\n R2: {} RMSE: {} MAE: {} MAPE: {} NRMSE: {}".format(me,mo,
                                                                              round(np.mean(R2test),2),
                                                                              round(np.mean(RMSEtest),0),
                                                                              round(np.mean(MAEtest),0),
                                                                              round(np.mean(MAPE),0),
                                                                              round(np.mean(NRMSE),2)))        
    print("\n")

-------------------- COMBINED --------------------
LR - M1:
 R2: 0.09 RMSE: 455.0 MAE: 343.0 MAPE: 1712.0 NRMSE: 0.99
LR - M5:
 R2: 0.23 RMSE: 421.0 MAE: 322.0 MAPE: 1831.0 NRMSE: 0.91


RF - M1:
 R2: 0.24 RMSE: 417.0 MAE: 316.0 MAPE: 1760.0 NRMSE: 0.9
RF - M5:
 R2: 0.23 RMSE: 419.0 MAE: 323.0 MAPE: 1932.0 NRMSE: 0.91


MLP - M1:
 R2: 0.24 RMSE: 417.0 MAE: 317.0 MAPE: 1833.0 NRMSE: 0.9
MLP - M5:
 R2: 0.24 RMSE: 417.0 MAE: 315.0 MAPE: 1796.0 NRMSE: 0.9


GB - M1:
 R2: 0.24 RMSE: 416.0 MAE: 314.0 MAPE: 1775.0 NRMSE: 0.9
GB - M5:
 R2: 0.23 RMSE: 419.0 MAE: 318.0 MAPE: 1881.0 NRMSE: 0.91


